In [1]:
#数据读取 为了方便大家更好的理解数据，我们重命名为中文
import numpy  as np
import pandas as pd
import os
# pandasql 可以把SQL代码写到这里，有些复杂的逻辑方便SQL写，并且日常应用也好参考,没有的安装下 pip install pandasql
from   pandasql import sqldf


data      = pd.read_csv("账户交易信息.csv", encoding="gbk")
train     = pd.read_csv("训练集标签.csv", encoding="gbk")
user_info = pd.read_csv("账户静态信息.csv", encoding="utf-8")
test      = pd.read_csv("test_dataset.csv", encoding="utf-8")
data = data.rename({
                    'jylsxh': '交易流水序号',
                    'zhdh': '账户代号',
                    'dfzh': '对方账号',
                    'jdbz': '借贷标志',
                    'jyje': '交易金额',
                    'jyye': '交易余额',
                    'dfhh': '对方行号',
                    'jyrq': '交易日期',
                    'jysj': '交易时间',
                    'jyqd': '交易渠道',
                    'zydh': '摘要代号',
                    'jdbj': '借贷标记',
                    'zhye': '账户余额',
                    'dfmccd': '对方名称长度'}, axis=1
                  )

train    = train.rename({'zhdh': '账户代号', 
                         'black_flag': '标签'}, axis=1
                       )

user_info =  user_info.rename({'zhdh': '账户代号',
                               'khrq':'开户日期', 
                               'xb': '性别', 
                             'khjgdh':'开户机构代号'}, axis=1
                              )

test    =  test.rename({'zhdh': '账户代号'},axis=1)

# #对数据进行聚合
# df = data.groupby(['账户代号','对方账号']).agg({'交易流水序号': pd.Series.nunique}).reset_index()
# print('聚合数据总数:\n',df.shape)

# # 剔除自己指向自己的节点
# df = df[df['账户代号']!=df['对方账号']]
# print('剔除后的数据:\n',df.shape)

In [2]:
# 异构网络构建
df1 = data[['账户代号','对方账号']]
df1.columns = ['账户代号','对方账号']


df2 = data[['账户代号','摘要代号']]
df2.columns = ['账户代号','对方账号']


# 关系合并
df = pd.concat([df1,df2])


# 对数据进行聚合
df = df.groupby(['账户代号','对方账号']).agg({'对方账号': ['count']}).reset_index()
df.columns=['账户代号','对方账号','CNT']

# 剔除自己指向自己的节点
print('剔除前的数据:\n',df.shape)

df = df[df['账户代号']!=df['对方账号']]
print('剔除后的数据:\n',df.shape)


剔除前的数据:
 (239495, 3)
剔除后的数据:
 (238808, 3)


In [3]:
# 关系数据转换成networkx的图格式
import networkx as nx
import matplotlib.pyplot as plt

# da = join_table[join_table['sim']>=0]
# print(da.shape)
da = df[['账户代号','对方账号']].values
G  = nx.Graph()
for num in range(len(da)):
    G.add_edge(str(da[num,0]),str(da[num,1]))
    

In [5]:
from node2vec import Node2Vec
from datetime import datetime

#加个运行时间的监控，这里是起点
start = datetime.now()

#参数设置
node2vec = Node2Vec(G,
                    dimensions=128, 
                    walk_length=10, 
                    p=0.8,
                    q=1.2,
                    num_walks=10, 
                    workers=4
                    ) 

#embedding 训练
model = node2vec.fit(window    = 8, 
                     min_count = 1, 
                     batch_words = 4
                    )

#加个运行时间的监控，这里是终点
end = datetime.now()
# 打印运行时间差
print(end - start)

Computing transition probabilities:   0%|          | 0/137592 [00:00<?, ?it/s]

Generating walks (CPU: 2): 100%|██████████| 3/3 [02:17<00:00, 45.85s/it]


0:16:48.664146


In [6]:
# 相似节点查找
model.wv.most_similar('1F77C9FDBDBE78D0',
                      topn = 20
                      ) 
                      
# 词列表
model.wv.index_to_key

# 词向量
model.wv.vectors

array([[ 0.3913144 , -0.0745106 ,  0.56893414, ..., -0.19531395,
        -0.22470807,  0.02920654],
       [-0.02110826, -0.04639608,  0.7374693 , ..., -0.4943545 ,
        -0.37653854,  0.36149612],
       [ 0.10577266, -0.5561032 ,  0.12941386, ...,  0.27673745,
        -0.05831195,  0.21263568],
       ...,
       [ 0.16044295, -0.38389388, -0.09044436, ..., -0.290499  ,
         0.05782848,  0.03880776],
       [-0.05336415, -0.08272731, -0.10765271, ..., -0.12042113,
         0.35115936, -0.00188111],
       [ 0.00277348, -0.20621154, -0.13479094, ..., -0.12463326,
         0.19796212, -0.09460197]], dtype=float32)

In [7]:
# 取出训练集里面的账户代号的的向量
zhanghao = pd.DataFrame(train[['账户代号']]).reset_index(drop=True)

embedded = pd.DataFrame([model.wv[x] for x in train['账户代号'].tolist()]).reset_index(drop=True)
embedded.columns = ['v_'+str(i) for i in range(0,128)]

embeddings = pd.concat([train,embedded],axis = 1)

embeddings

# 我们可以挑几个节点看看，是不是对齐的
pd.DataFrame([model.wv[x] for x in ['8D2F48B0BFC40AB2']])

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-0.185674,-0.297563,-0.117594,0.202553,0.754541,0.250272,-0.662021,0.303024,-0.31567,-0.080389,...,0.579538,0.290362,-0.287749,-0.550595,0.006674,0.563537,-0.205403,-0.551901,0.530594,-0.012123


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics  import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score,accuracy_score
from sklearn.model_selection import train_test_split


from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt


# 划分特征变量与目标变量
X = embeddings.drop(columns=['账户代号','标签'])
Y = embeddings['标签']

# 划分训练及测试集
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)


rf = RandomForestClassifier(n_estimators=300, 
                            max_depth=8,
                            verbose=1
                           )
rf.fit(x_train, y_train)

predict       = rf.predict(x_test) # 结果为0,1
predict_proba = rf.predict_proba(x_test) # 结果为概率值


# 找到0-1分类最佳的概率切分值
scores = []
thresholds = []

best_score = 0
best_threshold = 0


for threshold in np.arange(0.2, 0.6, 0.0001):
    preds = (predict_proba[:,1].reshape((-1)) > threshold).astype('int')
    m = f1_score(y_test, preds, average='macro')   
    scores.append(m)
    thresholds.append(threshold)
    if m > best_score:
        best_score = m
        best_threshold = threshold
print(f'threshold - {best_threshold:.08f}, best f1 - {best_score}')

preds = (predict_proba[:,1].reshape((-1)) >best_threshold).astype('int')

print('理论黑样本数:',str(int(240*0.25)),'\n')
print('预测黑样本数:',str(sum(preds)))


# 准确率 + F1值评估
print('准确率：',accuracy_score(y_test,predict),'\n')
print('F1值 ：',f1_score(y_test,preds, average='macro'))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


threshold - 0.27600000, best f1 - 0.8067632850241546
理论黑样本数: 60 

预测黑样本数: 74
准确率： 0.75 

F1值 ： 0.8067632850241546
